# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Camera calibration using chessboard images

In [18]:
# Load modules.
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0).
objp = np.zeros((6*9, 3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) # x, y coordinates

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space.
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images.
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    #print(fname)
    # Read in a calibration image.
    img = cv2.imread(fname)
    # Convert image to grayscale.
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners for grayscale images.
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)

    # If found, add object points, image points.
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners.
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
    
    # Save modified images
    path = str(fname)
    filename = path.split('/')[-1]
    new_path = './output_images/' + filename
    cv2.imwrite(new_path, img)

# Close all windows.
cv2.destroyAllWindows()

# Calculate camera calibration:
# camera matrix 'mtx'
# distortion coefficients 'dist'
# Position of camera in real world: rotation ('rvecs') and translation ('tvecs') vectors.
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

## Distortion correction